<a href="https://colab.research.google.com/github/lakshanravi/Conversational-RAG-app/blob/main/05.rag_application_langchain_openai/05_conversational_rag_langchain_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conversational RAG Application** with LangChain and Google GenAI LLM

In [1]:
# Install the necessary packages
!pip install langchain -qU
!pip install -U langchain langchain-google-genai
!pip install langchain-chroma -qU
!pip install langchain_community -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.

In [5]:
import os
from google.colab import userdata

### Initialize Google GenAI LLM

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Set OpenAI API key
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0
)

### Initialize Embedding Model

In [7]:
#for embedding the inputs
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

### Load PDF Document

In [8]:
!pip install pypdf -qU

In [9]:
#To upload the pdf file from my device. Through the imported pdf, we can store the farmscape data in vector DB.
from google.colab import files

# Upload file from your computer
uploaded = files.upload()


Saving FARMSCAP.pdf to FARMSCAP (1).pdf


In [10]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("/content/FARMSCAP.pdf")

docs = loader.load()

In [11]:
#show the number of pages in the pdf
len(docs)


7

### Split Documents into Chunks

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

In [13]:
 #splited into chunks
len(splits)

18

### Create Vector Store and Retriever

In [14]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [15]:
# Create a retriever from the vector store (we need retriver beacuse this need to retrive similar content based on user query)
retriever = vectorstore.as_retriever()

### Define Prompt Template

In [16]:
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt
#context is a response by the vector store. contain some chunks based on user input
system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

# Create the prompt template
#input given by the user
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [17]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [1]:
# Clean old broken installs
!pip uninstall -y langchain langchain-core langchain-community langchain-text-splitters langchain-chroma chromadb

# Install correct compatible stack
!pip install -U \
  langchain \
  langchain-core \
  langchain-community \
  langchain-text-splitters \
  langchain-chroma \
  langchain-google-genai \
  chromadb \
  google-generativeai


Found existing installation: langchain 1.2.10
Uninstalling langchain-1.2.10:
  Successfully uninstalled langchain-1.2.10
Found existing installation: langchain-core 1.2.13
Uninstalling langchain-core-1.2.13:
  Successfully uninstalled langchain-core-1.2.13
Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
Found existing installation: langchain-text-splitters 1.1.0
Uninstalling langchain-text-splitters-1.1.0:
  Successfully uninstalled langchain-text-splitters-1.1.0
Found existing installation: langchain-chroma 1.1.0
Uninstalling langchain-chroma-1.1.0:
  Successfully uninstalled langchain-chroma-1.1.0
Found existing installation: chromadb 1.5.0
Uninstalling chromadb-1.5.0:
  Successfully uninstalled chromadb-1.5.0
  Using cached langchain-1.2.10-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_core-1.2.13-py3-none-any.whl.metadata (4.4 kB)
  Using cached langchain_community-0.4.

### Create Retrieval-Augmented Generation (RAG) Chain

In [18]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, qa_chain)

ModuleNotFoundError: No module named 'langchain.chains'

In [28]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Convert retrieved docs into clean text
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [26]:
docs = retriever.invoke("who is codeprolk")
print(docs)


[Document(id='f72bb533-10d5-417b-8ede-abb4c5713add', metadata={'page': 0, 'producer': 'Microsoft® Word 2019', 'moddate': '2026-02-16T14:14:25+05:30', 'creator': 'Microsoft® Word 2019', 'page_label': '1', 'source': '/content/FARMSCAP.pdf', 'total_pages': 7, 'author': 'Ravindu Lakshan', 'creationdate': '2026-02-16T14:14:25+05:30'}, page_content='The system centralizes animal data, breeding records, feeding management, milk yield tracking, geofencing \nsecurity, and AI-based monitoring into one unified platform. \n \n1.2 Objectives \n• Digitize livestock farm operations \n• Improve productivity monitoring \n• Enable predictive yield forecasting \n• Enhance farm security through geofencing \n• Provide intelligent monitoring using AI'), Document(id='c7e40470-46c1-4668-bf35-1f6d49ac79da', metadata={'page_label': '3', 'moddate': '2026-02-16T14:14:25+05:30', 'total_pages': 7, 'producer': 'Microsoft® Word 2019', 'source': '/content/FARMSCAP.pdf', 'author': 'Ravindu Lakshan', 'creationdate': '20

### Invoke RAG Chain with Example Questions

In [31]:
#rag_chain.invoke({"input": "who is codeprolk"}) this way not work with gemini embeddings. below way works
response = rag_chain.invoke("give m e way for crete milk yeild record")
print(response)


To create a milk yield record, follow these steps:

1.  **Select animal:** Choose the specific animal for which you want to record the milk yield.
2.  **Enter milk quantity:** Input the amount of milk produced by the animal.
3.  **Submit record:** Confirm and submit the entered milk quantity.
4.  **System updates yield database:** The system will then update the milk yield database with this new record.


In [34]:
response = rag_chain.invoke("what is rag architecture")
print(response)

I don't know.


In [ ]:
response = rag_chain.invoke({"input": "what are the courses codeprolk offer"})
response["answer"]

'CodePRO LK offers a variety of free courses presented in Sinhala, catering to various proficiency levels from beginners to intermediates. Some key courses include Python GUI – Tkinter. In the future, CodePRO LK aims to expand its offerings to cover advanced topics like artificial intelligence, cybersecurity, and advanced data analytics.'

In [36]:
response = rag_chain.invoke( "can you list down")
print(response)

Please specify what you would like me to list down. I can provide information on:

*   **Feeding Management Module**
*   **Milk Yield Monitoring Module**
*   **Veterinary Service Location Module**
*   **Government Aid Information Module**
*   **Security & Geofencing Module**
*   **Animal Management Module**
*   **Breeding Management Module**
*   **Admin functionalities**
*   **User Management Module**

Let me know which module or section you are interested in!


## Add Chat History

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Define the contextualize system prompt
contextualize_system_prompt = (
    "using chat history and the latest user question, just reformulate question if needed and otherwise return it as is"
)

# Create the contextualize prompt template
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_prompt
)

#### Create History-Aware RAG Chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [ ]:
# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the history aware RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

#### Manage Chat Session History

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the store for session histories
store = {}

# Function to get the session history for a given session ID
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the conversational RAG chain with session history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

#### Invoke Conversational RAG Chain with Example Questions

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "who is codeprolk"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'CodePRO LK is a dynamic educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It offers a diverse range of technology-related courses in Sinhala, focusing on programming, data science, and machine learning. CodePRO LK aims to empower Sri Lankans with valuable skills in the tech industry through accessible and high-quality education. The platform continues to evolve and expand its offerings to support its mission of preparing learners for success in the global tech industry. Additionally, CodePRO LK engages its community through various events like webinars, live coding sessions, hackathons, and tech talks to provide networking opportunities and practical experience. The platform also collaborates with educational institutions, tech companies, and industry experts to enhance its content and resources, ensuring learners are well-prepared for real-world challenges.'

In [ ]:
store

{'101': InMemoryChatMessageHistory(messages=[HumanMessage(content='who is codeprolk'), AIMessage(content='CodePRO LK is a dynamic educational platform founded by Dinesh Piyasamara during the COVID-19 pandemic. It offers a diverse range of technology-related courses in Sinhala, focusing on programming, data science, and machine learning. CodePRO LK aims to empower Sri Lankans with valuable skills in the tech industry through accessible and high-quality education. The platform continues to evolve and expand its offerings to support its mission of preparing learners for success in the global tech industry. Additionally, CodePRO LK engages its community through various events like webinars, live coding sessions, hackathons, and tech talks to provide networking opportunities and practical experience. The platform also collaborates with educational institutions, tech companies, and industry experts to enhance its content and resources, ensuring learners are well-prepared for real-world chall

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "what is rag architecture"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'I\'m sorry, but I don\'t have information on "rag architecture." It seems to be a specific term or concept that is not related to the context provided about CodePRO LK. If you have any other questions or need clarification on a different topic, feel free to ask!'

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "what are the courses codeprolk offer"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

"CodePRO LK offers a variety of technology-related courses in Sinhala to empower learners with valuable skills in programming, data science, and machine learning. Some of the key courses offered by CodePRO LK include:\n\n1. Python GUI – Tkinter: This course covers the essentials of creating graphical user interfaces using Python's Tkinter library.\n\nCodePRO LK aims to cater to learners of all proficiency levels, from beginners to intermediates, ensuring that individuals at different stages can benefit from the courses. Additionally, the platform plans to expand its course offerings in the future to cover more advanced topics and emerging technologies such as artificial intelligence, cybersecurity, and advanced data analytics."

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "can you list down"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

"I apologize for the confusion earlier. Here is a list of some of the courses offered by CodePRO LK:\n\n1. Python GUI – Tkinter: This course covers the essentials of creating graphical user interfaces using Python's Tkinter library.\n\nPlease note that CodePRO LK aims to expand its course offerings in the future to cover more advanced topics and emerging technologies such as artificial intelligence, cybersecurity, and advanced data analytics."